# Module 5 Homework - Let's Get Lunch

Suppose we are looking for a good place for lunch on our next trip to La Crosse. Let's collect some data on nearby restaurants.

## <font color="red"> Problem 1 </font>

Goto yelp.com and perform a search with the following parameters

* **Find** Restaurants
* **Near** La Crosse, WI

**Tasks**
1. Copy the resulting web address below and determine the how the     specified search terms related to the resulting address
2. Use requests and Beautiful Soap to download the content of the front page.

In [1]:
# Install if needed
!pip install composable
!pip install composablesoup
# Check for upgrade is already installed
!pip install composable --upgrade
!pip install composablesoup --upgrade

Requirement already up-to-date: composable in /home/sbada9048/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages (0.2.5)
Requirement already up-to-date: composablesoup in /home/sbada9048/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages (0.2.4)


In [2]:
# Get and process the Yelp search
from composable import pipeable
from composable.strict import map, filter
from composablesoup import find, find_all, get_text, has_attr
from composablesoup.soup import find_parent, parents, children, find_previous_sibling, find_previous_siblings, find_next_sibling, find_next_siblings, find_previous_sibling
from composable.sequence import to_list, head
from composable.string import strip
from composable import from_toolz as tlz

In [3]:
import requests
from bs4 import BeautifulSoup
s = requests.Session()
r = s.get('https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI')
yelp = BeautifulSoup(r.content, 'html.parser')

## <font color="red"> Problem 2 </font>

We want to grab the restaurant's name.

1. Use Inspect Element to determine the tags/classes for each of the elements listed above.  
2. Note that all the business information is contained in a`div` that contains a class similar to  `<div class=" ... businessName__09f24__3Wql2 ...">`.  You will need to use a regular expression to match the `businessName` in the class (see lecture 5.3).
3. Write expressions/functions to pull out the name of each restaurant.  
    * Note: The business name is in an unnamed tag, you will need to navigate to the using searches and/or relationship
 
**Confirm that there is an extra restaurant in the list (e.g. 11-12 instead of 10). This is due to an advertisement/sponsered links, which we want to ignore.**

In [4]:
(yelp)

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/no-js/,"js");</script><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="en-US" http-equiv="Content-Language"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><link content="#c41200" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/97ea654d5227/assets/img/logos/yelp_burst.svg" rel="mask-icon" sizes="any"/><link href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/118ff475a341/assets/img/logos/favicon.ico" rel="shortcut icon"/><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script async="" src="https://www.google-analytics.com/analy

In [5]:
import re
resturant_finder = re.compile('link__09f24.*')
(yelp
 >> find_all('a', attrs = {'class': resturant_finder})
 >> map(get_text)
)

['Yelp',
 'For Businesses',
 'Write a Review',
 'See all',
 'See all',
 'Recommended',
 '',
 'Pappi’s Taqueria y Mas',
 'Food Trucks',
 'Mexican',
 'more',
 '',
 'The Waterfront Restaurant & Tavern',
 'American (New)',
 'Seafood',
 'Steakhouses',
 'more',
 '',
 'Restore Public House',
 'American (Traditional)',
 'more',
 '',
 'Lovechild Restaurant',
 'American (New)',
 'more',
 '',
 'Buzzard Billy’s',
 'American (Traditional)',
 'Cajun/Creole',
 'more',
 '',
 'The Charmant',
 'French',
 'Cocktail Bars',
 'more',
 '',
 'The Freighthouse Restaurant',
 'Seafood',
 'Steakhouses',
 'Desserts',
 'more',
 '',
 'Howie’s on La Crosse',
 'American (New)',
 'Pubs',
 'more',
 '',
 'Schuby’s Neighborhood Butcher',
 'Butcher',
 'Delis',
 'Caterers',
 'more',
 '',
 'The Crow',
 'American (Traditional)',
 'Gastropubs',
 'more',
 '',
 'Piggy’s Restaurant',
 'Steakhouses',
 'Seafood',
 'Sandwiches',
 'more',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '',
 'Help us improve.',
 'Virtual Cooking Clas

## <font color="red"> Problem 3 </font>

Since we picked up extra information, we will need to be clever about identifying the information block for each restaurant.  Note that all of the actual search results (but not sponsered links) start with the ranking (e.g. `11.`).  Use the following steps to get a list that contains the information for each restaurant other than the adds.

1. Start by finding the ranking of the restaurant (1., 2., etc.). **Hint:** You will need to use regular expression to match the text of the tag (see lecture 5.3).
2. Now search for a parent of the above tags that surrounds all of the restaurant information.  You will want to use the `find_parent` method on each of the tags from **1.**.  **Hint:** Look through each of the `div` tags that contain the ranking, looking for a meaningful tag name to match with a regular expression.

The resulting list will be the starting point for gathering all of the information.

In [49]:
wrapper_finder = re.compile('mainAttributes__09f24__26-vh')
previous_parent_finder = re.compile('scrollablePhotos__09f24__1PpB8')
find_list = (yelp
 >> find_all('div', attrs = {'class' : wrapper_finder})
 >> map(find_parent(name='div', attrs = {'class' : previous_parent_finder}))
)
find_list

[<div class="scrollablePhotos__09f24__1PpB8 arrange__09f24__AiSIM border-color--default__09f24__R1nRO"><div class="mainAttributes__09f24__26-vh arrange-unit__09f24__1gZC1 arrange-unit-fill__09f24__O6JFU border-color--default__09f24__R1nRO"><div class="border-color--default__09f24__R1nRO"><div class="border-color--default__09f24__R1nRO"><div class="businessName__09f24__3Wql2 display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO"><div class="border-color--default__09f24__R1nRO"><h4 class="heading--h4__09f24__2ijYq alternate__09f24__39r7c"><span class="text__09f24__2tZKC text-color--black-regular__09f24__1QxyO text-align--left__09f24__3Drs0 text-weight--bold__09f24__WGVdT text-size--inherit__09f24__2rwpp"><a class="link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--inherit__09f24__2Uj95" href="/adredir?ad_business_id=KJamo9Yl6A98VDNI_V0S9w&amp;campaign_id=okhiVFYG8-ua2Z36eBBn3g&amp;click_origin=search_results&amp;placement=above_search&amp;placement_slot=0&a

## <font color="red"> Problem 4 </font>

Write expressions/functions to gather each of the following pieces of information for each of the restaurants.

#### Restaurant Name

In [50]:
resturant_finder = re.compile('link__09f24.*')
find_resturant = pipeable(lambda List: [line >> find('a', attrs = {'class': resturant_finder}) for line in List])
resturants = (find_list
 >> find_resturant
 >> map(get_text)
)
resturants

['Pappi’s Taqueria y Mas',
 'The Waterfront Restaurant & Tavern',
 'Restore Public House',
 'Lovechild Restaurant',
 'Buzzard Billy’s',
 'The Charmant',
 'The Freighthouse Restaurant',
 'Howie’s on La Crosse',
 'Schuby’s Neighborhood Butcher',
 'The Crow',
 'Piggy’s Restaurant']

#### Rating

In [51]:
rating_finder = re.compile('i-stars__09f24__1T6rz')
find_rating = pipeable(lambda List: [line >> find('div', attrs = {'class': rating_finder}) for line in List])
ratings = (find_list
 >> find_rating
 >> map(tlz.get('aria-label'))
)
ratings

['4 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4.5 star rating',
 '4 star rating',
 '4 star rating',
 '4.5 star rating',
 '3.5 star rating',
 '4 star rating']

#### Address

In [54]:
address_finder = re.compile('raw__09f24__3Obuy')
find_address = pipeable(lambda List: [line >> find('span', attrs = {'class': address_finder}) for line in List])
addresses = (find_list
 >> find_address
 >> map(get_text)
)
addresses

['83 Buol Rd',
 '328 Front St S',
 '1810 State St',
 '300 3rd St S',
 '222 Pearl St',
 '101 State St',
 '107 Vine St',
 '1128 La Crosse St',
 '321 State St',
 '100 3rd St S',
 '501 Front St S']

#### Review Count

#### Category

**Examples:** `['American (New)', 'Seafood', 'Steakhouses']` becomes `'American (New);Seafood;Steakhouses'`

## <font color="red">  Problem 4 </font>

Package all of the expressions in a function that takes a url as input and returns the table of information.  Use a `def` statement and put the above helper functions in the body of the main function.  Test this function on the front page of the search.

## <font color="red">  Problem 5 </font>

Now perform a linear search to grab all of the information on restaurants in La Crosse.  You will need to

1. Inspect the url for the first, second, etc. pages to determine the pattern.
2. Create a list of urls for all search results.
3. Get the info from all pages and use your function from the last problem to parse the data into a list.
4. Write the results to a csv file. **Hint:** Use `'a'` (append) instead of `'w'` write on all after the first/

## <font color="red">  Bonus Problem </font>

See if you can also grab the latitude and longitude of each result.